<a href="https://colab.research.google.com/github/mzignis/loan_prediction/blob/master/explore_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
HOME = '/content/drive/My Drive/Colab Notebooks/projects/loan_prediction/loan_prediction'
%cd $HOME

/content/drive/My Drive/Colab Notebooks/projects/loan_prediction/loan_prediction


In [0]:
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler

In [3]:
df = pd.read_csv('data/train.csv')
df.shape

(614, 13)

In [4]:
df.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [5]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [6]:
for column_name in df.columns[1:]:
    print(f'{column_name}: {df[column_name].dtype}')
    unique_values = df[column_name].unique()
    unique_values = unique_values.astype(str)
    if unique_values.shape[0] > 5:
        unique_values = unique_values.tolist()[:5] + ['...']
    print(f"[{', '.join(unique_values)}]")
    print()

Gender: object
[Male, Female, nan]

Married: object
[No, Yes, nan]

Dependents: object
[0, 1, 2, 3+, nan]

Education: object
[Graduate, Not Graduate]

Self_Employed: object
[No, Yes, nan]

ApplicantIncome: int64
[5849, 4583, 3000, 2583, 6000, ...]

CoapplicantIncome: float64
[0.0, 1508.0, 2358.0, 4196.0, 1516.0, ...]

LoanAmount: float64
[nan, 128.0, 66.0, 120.0, 141.0, ...]

Loan_Amount_Term: float64
[360.0, 120.0, 240.0, nan, 180.0, ...]

Credit_History: float64
[1.0, 0.0, nan]

Property_Area: object
[Urban, Rural, Semiurban]

Loan_Status: object
[Y, N]



In [0]:
categorical_columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 
                       'Credit_History', 'Property_Area', 'Loan_Status']
numerical_columns = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']

In [0]:
for column in categorical_columns:
    most_common_value = df[column].value_counts().index[0]
    df[column].fillna(most_common_value, inplace=True)

In [9]:
applicant_income_mean = df['ApplicantIncome'].mean()
coapplicant_income_mean = df['CoapplicantIncome'].mean()
applicant_income_mean, coapplicant_income_mean

(5403.459283387622, 1621.245798027101)

In [0]:
for column in numerical_columns:
    df[column].fillna(df[column].mean(), inplace=True)

In [11]:
df.isnull().values.any()

False

In [0]:
categorical_columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 
                       'Credit_History', 'Property_Area', 'Loan_Status']
numerical_columns = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']

def fill_na(data_frame, categorical_columns=categorical_columns, numerical_columns=numerical_columns):
    df = data_frame.copy()
    for column in df.columns:
        if column in categorical_columns:
            most_common_value = df[column].value_counts().index[0]
            df[column].fillna(most_common_value, inplace=True)
        elif column in numerical_columns:
            df[column].fillna(df[column].mean(), inplace=True)
        else:
            pass

    return df

df = pd.read_csv('data/train.csv')
df = fill_na(df)

In [13]:
def encode_categorical_values(data_frame, categorical_columns=categorical_columns, y_column='Loan_Status'):
    df = pd.DataFrame()
    for column in categorical_columns:
        if column == y_column:
            encoder = LabelEncoder()
            encoded_column = encoder.fit_transform(data_frame[column].values.reshape(-1, 1))
            df[column] = encoded_column  
        else:
            encoder = OneHotEncoder()
            encoded_columns = encoder.fit_transform(data_frame[column].values.reshape(-1, 1))
            for ii in range(encoded_columns.toarray().shape[1]):
                df[f'{column}_{encoder.categories_[0][ii]}'] = encoded_columns.toarray()[:, ii]

    return df


df = pd.read_csv('data/train.csv')
df = fill_na(df)
df_categorical = encode_categorical_values(df)
df_categorical.head()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Credit_History_0.0,Credit_History_1.0,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1
1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0
2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1
3,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1
4,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1


In [14]:
def scale_numeric_values(data_frame, numerical_columns=numerical_columns):
    df = pd.DataFrame()
    for column in numerical_columns:
        scaller = MinMaxScaler()
        scalled_column = scaller.fit_transform(data_frame[column].values.reshape(-1, 1))
        df[column] = scalled_column.ravel()

    return df


df = pd.read_csv('data/train.csv')
df = fill_na(df)
df_numerical = scale_numeric_values(df)
df_numerical.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term
0,0.070489,0.000000,0.198860,0.74359
1,0.054830,0.036192,0.172214,0.74359
2,0.035250,0.000000,0.082489,0.74359
3,0.030093,0.056592,0.160637,0.74359
4,0.072356,0.000000,0.191027,0.74359


In [0]:
def preapare_data(csv_path, categorical_columns=categorical_columns, numerical_columns=numerical_columns, 
                  y_column='Loan_Status'):
    
    df = pd.read_csv(csv_path)
    df = fill_na(df, categorical_columns, numerical_columns)
    df_categorical = encode_categorical_values(df, categorical_columns, y_column)
    df_numerical = scale_numeric_values(df, numerical_columns)
    df = pd.concat([df_categorical, df_numerical], axis=1)

    if y_column in df.columns:
        print('lol')
        return df.drop(columns=[y_column]).values, df[y_column].values
    else:
        return df


In [0]:
categorical_columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 
                       'Credit_History', 'Property_Area', 'Loan_Status']
numerical_columns = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']

In [17]:
X_train, y_train = preapare_data('data/train.csv')
X_train.shape, y_train.shape

lol


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


((614, 21), (614,))

In [0]:
train_data = {
    'X_train': X_train,
    'y_train': y_train,
}

with open('data/train.p', 'wb') as pf:
    pickle.dump(train_data, pf)

In [19]:
categorical_columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 
                       'Credit_History', 'Property_Area']
X_test = preapare_data('data/test.csv', categorical_columns)
X_test.shape

(367, 21)

In [0]:
test_data = {
    'X_test': X_test,
}

with open('data/test.p', 'wb') as pf:
    pickle.dump(test_data, pf)